In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [ ]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [ ]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [ ]:

# from langchain_community.document_loaders import PyPDFLoader
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        pages += loader.load()
    # break # too much time, only load one file
pages

# loader = PyPDFLoader(os.path.join(dir, '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf'))
# pages += loader.load()
# pages = [pages[9]]
# pages

In [ ]:
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=5)
chunk_list = graph_builder.graph_build(pages)
chunk_list

In [ ]:
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for chunk in chunk_list:
#     doc = chunk['chunk_doc']
#     content = doc.page_content
#     curr_num = num_tokens_from_string(content)
#     tokens_num += num_tokens_from_string(content)
# tokens_num

In [ ]:
graph_builder._get_combined_chunks(chunk_list)

In [ ]:

graph_documents = graph_builder.get_graph_from_llm(llm, chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

In [9]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [ ]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

In [11]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)